In [20]:
from pyspark.sql import SparkSession
#from pyspark.sql.types import StructField, StructType, StringType, IntegerType, LongType
#from pyspark.sql.functions import from_json,col, to_timestamp, date_format, concat, count
import pyspark.sql.functions as F

def get_spark_session():
    spark = SparkSession \
        .builder \
        .appName("cassandra") \
        .config("spark.streaming.stopGracefullyOnShutdown", True) \
        .config("spark.jars.packages", "com.datastax.spark:spark-cassandra-connector_2.12:3.4.1") \
        .config("spark.cassandra.connection.host", "events-db") \
        .config("spark.cassandra.connection.port","9042")\
        .config("spark.cassandra.auth.username", "cassandra") \
        .config("spark.cassandra.auth.password", "cassandra") \
        .config("spark.sql.shuffle.partitions", 4) \
        .master("spark://spark-master:7077") \
        .getOrCreate()
    return spark


def read_bucket(keyspace_name, table_name, bucket_name):
    bucket_df = spark.read\
            .format("org.apache.spark.sql.cassandra")\
            .options(table=table_name, keyspace=keyspace_name)\
            .load()\
            .where(col("bucket")==bucket_name)
    return bucket_df

In [21]:
spark = get_spark_session()

In [22]:
bucket_df = read_bucket("events", "events_raw", "202311250000")
#sessions_count = bucket_df.count()

## agg by dimensions

In [30]:
dimensions = ["category", "applicationProtocol", "applicationName","domain","deviceType","networkType","contentType"]
bucket_df.groupBy(dimensions)\
    .agg(F.count("*").alias("ccc"),\
    F.sum("bytesFromClient").alias("bytesFromClient"),\
    F.sum("bytesToClient").alias("bytesToClient"),\
    F.sum("bytesFromServer").alias("bytesFromServer"),\
    F.sum("bytesToServer").alias("bytesToServer"),\
    F.sum("lostBytesClient").alias("lostBytesClient"),\
    F.sum("lostBytesServer").alias("lostBytesServer"),\
    F.sum("srttMsClient").alias("srttMsClient"),\
    F.sum("srttMsServer").alias("srttMsServer"))\
    .show(4)

+--------+-------------------+---------------+----------------+-----------+-----------+-----------+---+---------------+-------------+---------------+-------------+---------------+---------------+------------+------------+
|category|applicationProtocol|applicationName|          domain| deviceType|networkType|contentType|ccc|bytesFromClient|bytesToClient|bytesFromServer|bytesToServer|lostBytesClient|lostBytesServer|srttMsClient|srttMsServer|
+--------+-------------------+---------------+----------------+-----------+-----------+-----------+---+---------------+-------------+---------------+-------------+---------------+---------------+------------+------------+
|    voip|              quick|       whatsapp|app.whatsapp.com|  iphone 12|         4g|      quick|133|         405021|      1039825|        1039825|       405021|              1|              6|       35639|       38837|
|    voip|              quick|       whatsapp|app.whatsapp.com|  iphone 12|         5g|      quick|117|         

## agg by subscriber

In [31]:
dimensions = ["subscriberID"]
bucket_df.groupBy(dimensions)\
    .agg(F.count("*").alias("ccc"),\
    F.sum("bytesFromClient").alias("bytesFromClient"),\
    F.sum("bytesToClient").alias("bytesToClient"),\
    F.sum("bytesFromServer").alias("bytesFromServer"),\
    F.sum("bytesToServer").alias("bytesToServer"),\
    F.sum("lostBytesClient").alias("lostBytesClient"),\
    F.sum("lostBytesServer").alias("lostBytesServer"),\
    F.sum("srttMsClient").alias("srttMsClient"),\
    F.sum("srttMsServer").alias("srttMsServer"))\
    .show(4)

+-------------+---+---------------+-------------+---------------+-------------+---------------+---------------+------------+------------+
| subscriberID|ccc|bytesFromClient|bytesToClient|bytesFromServer|bytesToServer|lostBytesClient|lostBytesServer|srttMsClient|srttMsServer|
+-------------+---+---------------+-------------+---------------+-------------+---------------+---------------+------------+------------+
|2016828382090| 33|         220708|       339893|         339893|       220708|              0|              0|        9085|        9896|
|2016828854411| 22|          88676|       356435|         356435|        88676|              0|              0|        6446|        5848|
|2016828850135| 24|         139559|       298831|         298831|       139559|             20|              0|        6908|        7706|
|2016828107431| 21|          75490|       215228|         215228|        75490|              0|              0|        5626|        5746|
+-------------+---+---------------

## agg by location

In [33]:
dimensions = ["locationID"]
bucket_df.groupBy(dimensions)\
    .agg(F.count("*").alias("ccc"),\
    F.sum("bytesFromClient").alias("bytesFromClient"),\
    F.sum("bytesToClient").alias("bytesToClient"),\
    F.sum("bytesFromServer").alias("bytesFromServer"),\
    F.sum("bytesToServer").alias("bytesToServer"),\
    F.sum("lostBytesClient").alias("lostBytesClient"),\
    F.sum("lostBytesServer").alias("lostBytesServer"),\
    F.sum("srttMsClient").alias("srttMsClient"),\
    F.sum("srttMsServer").alias("srttMsServer"))\
    .show(4)

+------------+---+---------------+-------------+---------------+-------------+---------------+---------------+------------+------------+
|  locationID|ccc|bytesFromClient|bytesToClient|bytesFromServer|bytesToServer|lostBytesClient|lostBytesServer|srttMsClient|srttMsServer|
+------------+---+---------------+-------------+---------------+-------------+---------------+---------------+------------+------------+
|171a9ffb8a20|204|         753559|      1317422|        1317422|       753559|             89|             45|       54926|       56172|
|7a264a01158a|204|         927710|      1825999|        1825999|       927710|            636|            532|       58644|       56611|
|743de1396a40|212|         635221|      1226056|        1226056|       635221|             42|             30|       58708|       58607|
|620ad605906f|178|         613630|       925699|         925699|       613630|            792|            230|       50013|       44749|
+------------+---+---------------+-------

## aggregation by all

In [34]:
dimensions = ["subscriberID", "locationID", "category", "applicationProtocol", "applicationName","domain","deviceType","networkType","contentType"]
bucket_df.groupBy(dimensions)\
    .agg(F.count("*").alias("ccc"),\
    F.sum("bytesFromClient").alias("bytesFromClient"),\
    F.sum("bytesToClient").alias("bytesToClient"),\
    F.sum("bytesFromServer").alias("bytesFromServer"),\
    F.sum("bytesToServer").alias("bytesToServer"),\
    F.sum("lostBytesClient").alias("lostBytesClient"),\
    F.sum("lostBytesServer").alias("lostBytesServer"),\
    F.sum("srttMsClient").alias("srttMsClient"),\
    F.sum("srttMsServer").alias("srttMsServer"))\
    .show(4)

+-------------+------------+----------------+-------------------+---------------+----------------+-------------+-----------+-----------+---+---------------+-------------+---------------+-------------+---------------+---------------+------------+------------+
| subscriberID|  locationID|        category|applicationProtocol|applicationName|          domain|   deviceType|networkType|contentType|ccc|bytesFromClient|bytesToClient|bytesFromServer|bytesToServer|lostBytesClient|lostBytesServer|srttMsClient|srttMsServer|
+-------------+------------+----------------+-------------------+---------------+----------------+-------------+-----------+-----------+---+---------------+-------------+---------------+-------------+---------------+---------------+------------+------------+
|2016828476442|d26efc1df266|            voip|              quick|       whatsapp|app.whatsapp.com|    iphone 12|         4g|      quick|  1|          29831|        63545|          63545|        29831|              0|       